# Iterating Over Array

https://numpy.org/doc/stable/reference/arrays.nditer.html

The iterator object nditer, introduced in NumPy 1.6, provides many flexible ways to visit all the elements of one or more arrays in a systematic fashion. 

This page introduces some basic ways to use the object for computations on arrays in Python, then concludes with how one can accelerate the inner loop in Cython. 

Since the Python exposure of nditer is a relatively straightforward mapping of the C array iterator API, these ideas will also provide help working with array iteration from C or C++.

NumPy package contains an iterator object numpy.nditer. 

It is an efficient multidimensional iterator object using which it is possible to iterate over an array. 

Each element of an array is visited using Python’s standard Iterator interface.

Let us create a 3X4 array using arange() function and iterate over it using nditer.

### Example

In [3]:
import numpy as np
a = np.arange(0,60,5)
# for (i = 0; i < 60; i += 5)
print('a:')
print(a)

a = a.reshape(3,4)
print('\na.reshape (3, 4):')
print(a)

print('\nModified array is:')
for x in np.nditer(a):
   print (x, end=" ")

a:
[ 0  5 10 15 20 25 30 35 40 45 50 55]

a.reshape (3, 4):
[[ 0  5 10 15]
 [20 25 30 35]
 [40 45 50 55]]

Modified array is:
0 5 10 15 20 25 30 35 40 45 50 55 

### Example

The order of iteration is chosen to match the memory layout of an array, without considering a particular ordering. 

This can be seen by iterating over the transpose of the above array.

In [16]:
import numpy as np 
a = np.arange(0,60,5) 
# for (i = 0; i < 60; i += 5)
print('a:')
print(a)  

a = a.reshape(3,4)
print ('\na.shape:', a.shape)
print('\na:')
print(a)

print ('\nTranspose of the original array is (4, 3):') 
b = a.T 
print ('\nb.shape:', b.shape) 
print (b)   
   
print('\nModified array is:') 
for x in np.nditer(b): 
   print (x, end=" ")

a:
[ 0  5 10 15 20 25 30 35 40 45 50 55]

a.shape: (3, 4)

a:
[[ 0  5 10 15]
 [20 25 30 35]
 [40 45 50 55]]

Transpose of the original array is (4, 3):

b.shape: (4, 3)
[[ 0 20 40]
 [ 5 25 45]
 [10 30 50]
 [15 35 55]]

Modified array is:
0 5 10 15 20 25 30 35 40 45 50 55 

## Iteration Order

If the same elements are stored using F-style (Fortran-style) order, the iterator chooses the more efficient way of iterating over an array.

## Note:
- C-Style is row first
- F-Style is column first.

### Example

In [6]:
import numpy as np
a = np.arange(0,60,5)
print('a:')
print(a)
a = a.reshape(3,4)
print('a.shape:', a.shape)
print('\nOriginal array is:')
print(a)

print('\nTranspose of the original array is:')
b = a.T
print('\nb.shape:', b.shape)
print('\nb:')
print(b)

print('\nSorted in C-style order:')
c = b.copy(order = 'C')
print('c:')
print(c)

for x in np.nditer(c):
   print(x, end=" ")
print()

print('\nSorted in F-style order:')
c = b.copy(order = 'F')
print('c:')
print(c)

for x in np.nditer(c):
   print(x,end=" ")

a:
[ 0  5 10 15 20 25 30 35 40 45 50 55]
a.shape: (3, 4)

Original array is:
[[ 0  5 10 15]
 [20 25 30 35]
 [40 45 50 55]]

Transpose of the original array is:

b.shape: (4, 3)

b:
[[ 0 20 40]
 [ 5 25 45]
 [10 30 50]
 [15 35 55]]

Sorted in C-style order:
c:
[[ 0 20 40]
 [ 5 25 45]
 [10 30 50]
 [15 35 55]]
0 20 40 5 25 45 10 30 50 15 35 55 

Sorted in F-style order:
c:
[[ 0 20 40]
 [ 5 25 45]
 [10 30 50]
 [15 35 55]]
0 5 10 15 20 25 30 35 40 45 50 55 

### Example

We can to force nditer object to use a specific order by explicitly mentioning it.

In [25]:
import numpy as np 
a = np.arange(0,60,5) 
a = a.reshape(3,4) 

print ('Original array is:') 
print (a) 
print ()  

print ('\nSorted in C-style order:') 
for x in np.nditer(a, order = 'C'): 
   print (x, end=" ")  

print ()

print ('\nSorted in F-style order:') 
for x in np.nditer(a, order = 'F'): 
   print (x, end=" ")

Original array is:
[[ 0  5 10 15]
 [20 25 30 35]
 [40 45 50 55]]


Sorted in C-style order:
0 5 10 15 20 25 30 35 40 45 50 55 

Sorted in F-style order:
0 20 40 5 25 45 10 30 50 15 35 55 

## Modifying Array Values

The nditer object has another optional parameter called op_flags. 

Its default value is read-only, but can be set to read-write or write-only mode. 

This will enable modifying array elements using this iterator.

### Example

In [28]:
import numpy as np
a = np.arange(0,60,5)
a = a.reshape(3,4)
print ('Original array is:')
print (a)
print ()

for x in np.nditer(a, op_flags = ['readwrite']):
   x[...] = 2*x

print ('\nModified array is:')
print (a)

with np.nditer(a, op_flags=['readwrite']) as it:
   for x in it:
       x[...] = 2 * x       
print ('\nModified array is:')
print (a)

Original array is:
[[ 0  5 10 15]
 [20 25 30 35]
 [40 45 50 55]]


Modified array is:
[[  0  10  20  30]
 [ 40  50  60  70]
 [ 80  90 100 110]]

Modified array is:
[[  0  20  40  60]
 [ 80 100 120 140]
 [160 180 200 220]]


## External Loop

The nditer class constructor has a ‘flags’ parameter, which can take the following values

| No  | Parameter and Description     |
|:----|:------------------------------|
| 1   | c_index<br>C_order index can be tracked|
| 2   | f_index<br>Fortran_order index is tracked|
| 3   | multi-index<br>Type of indexes with one per iteration can be tracked|
| 4   | external_loop<br>Causes values given to be one-dimensional arrays with multiple values instead of zero-dimensional array|

### Example
In the following example, one-dimensional arrays corresponding to each column is traversed by the iterator.

In [31]:
import numpy as np 
a = np.arange(0,60,5) 
a = a.reshape(3,4) 

print('Original array is:') 
print(a) 
print()  

print('Modified array is:') 
for x in np.nditer(a, flags = ['external_loop'], order = 'F'):
   print(x, end=" ")

Original array is:
[[ 0  5 10 15]
 [20 25 30 35]
 [40 45 50 55]]

Modified array is:
[ 0 20 40] [ 5 25 45] [10 30 50] [15 35 55] 

## Buffering the Array Elements

When forcing an iteration order, the external loop option may provide the elements in smaller chunks because the elements can’t be visited in the appropriate order with a constant stride. 

When writing C code, this is generally fine, however in pure Python code this can cause a significant reduction in performance.

By enabling buffering mode, the chunks provided by the iterator to the inner loop can be made larger, significantly reducing the overhead of the Python interpreter. 

In the example forcing Fortran iteration order, the inner loop gets to see all the elements in one go when buffering is enabled.

In [37]:
import numpy as np

a = np.arange(6).reshape(2,3)
print('a.shape:', a.shape)
print('a:')
print(a)

print('\nexternal Fortran (Row First) loop:')
for x in np.nditer(a, flags=['external_loop'], order='F'):
    print(x, end=' ')

print()
print('\nexternal Fortran (Row First) buffered 1-D loop:')
for x in np.nditer(a, flags=['external_loop','buffered'], order='F'):
    print(x, end=' ')

a.shape: (2, 3)
a:
[[0 1 2]
 [3 4 5]]

external Fortran (Row First) loop:
[0 3] [1 4] [2 5] 

external Fortran (Row First) buffered 1-D loop:
[0 3 1 4 2 5] 

## Broadcasting Iteration

If two arrays are broadcastable, a combined nditer object is able to iterate upon them concurrently. 

Assuming that an array a has dimension 3X4, and there is another array b of dimension 1X4, the iterator of following type is used (array b is broadcast to size of a).

### Example

In [53]:
import numpy as np 
a = np.arange(0,60,5) 
a = a.reshape(3,4) 
print ('a.shape:', a.shape)

print('First array is:') 
print(a) 
print()  

print('Second array is:') 
b = np.array([1, 2, 3, 4], dtype = int) 
print(b)  
print() 

print('Modified array is:') 
for x,y in np.nditer([a,b]): 
   print(f"{x}:{y}".format(field1=x, field2=y), end= " ")

a.shape: (3, 4)
First array is:
[[ 0  5 10 15]
 [20 25 30 35]
 [40 45 50 55]]

Second array is:
[1 2 3 4]

Modified array is:
0:1 5:2 10:3 15:4 20:1 25:2 30:3 35:4 40:1 45:2 50:3 55:4 